In [8]:
import numpy as np

In [9]:
class RNN:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.001, steps=5):
        self.input_dim, self.hidden_dim, self.output_dim = input_dim, hidden_dim, output_dim
        self.lr, self.steps = lr, steps

        # Initialize weights and biases
        self.Wxh = np.random.randn(hidden_dim, input_dim) * 0.01
        self.Whh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Why = np.random.randn(output_dim, hidden_dim) * 0.01
        self.bh, self.by = np.zeros((hidden_dim, 1)), np.zeros((output_dim, 1))

    def forward(self, inputs):
        h = np.zeros((self.hidden_dim, 1))
        h_states, outputs = [], []

        for t in range(self.steps):
            h = np.tanh(self.Wxh @ inputs[t] + self.Whh @ h + self.bh)
            outputs.append(self.Why @ h + self.by)
            h_states.append(h)

        return h_states, outputs

    def backward(self, inputs, targets, h_states, outputs):
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        dh_next = np.zeros((self.hidden_dim, 1))

        for t in reversed(range(self.steps)):
            dy = outputs[t] - targets[t]
            dWhy += dy @ h_states[t].T
            dby += dy

            dh = self.Why.T @ dy + dh_next
            dh_raw = (1 - h_states[t] ** 2) * dh
            dbh += dh_raw
            dWxh += dh_raw @ inputs[t].T
            dWhh += dh_raw @ (h_states[t - 1] if t > 0 else np.zeros_like(h_states[0])).T
            dh_next = self.Whh.T @ dh_raw

        # Gradient clipping and weight update
        for param, dparam in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by], 
                                 [dWxh, dWhh, dWhy, dbh, dby]):
            np.clip(dparam, -5, 5, out=dparam)
            param -= self.lr * dparam

    def train(self, data, labels, epochs=100):
        for epoch in range(epochs):
            loss = 0
            for inputs, targets in zip(data, labels):
                h_states, outputs = self.forward(inputs)
                self.backward(inputs, targets, h_states, outputs)
                loss += np.sum((outputs[-1] - targets[-1]) ** 2) / 2
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Loss {loss:.4f}")


In [14]:
steps = 5
input_dim = 3
hidden_dim = 4
output_dim = 2

data = [np.random.randn(steps, input_dim, 1) * 0.1 for _ in range(100)]
labels = [np.random.randn(steps, output_dim, 1) * 0.1 for _ in range(100)]

rnn = RNN(input_dim, hidden_dim, output_dim)
rnn.train(data, labels, epochs=50)

Epoch 0: Loss 0.9017
Epoch 10: Loss 0.9027
Epoch 20: Loss 0.9027
Epoch 30: Loss 0.9027
Epoch 40: Loss 0.9027
